In [1]:
import os                                                                        
import time                                                                      
import json
import numpy as np
import pandas as pd
import dask as dd
from timeit import default_timer as timer
import gcsfs, json
from katana import remote
from katana.remote import import_data
from datetime import datetime
from dask.dataframe import from_pandas
from dask.dataframe import to_numeric
# os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"

In [2]:
from dask.distributed import Client, progress
client = Client("10.0.0.101:8786")

In [3]:
print(dd.config.get("distributed.client"))

{'heartbeat': '5s', 'scheduler-info-interval': '2s', 'security-loader': None, 'preload': [], 'preload-argv': []}


In [4]:
import dask.bag as db
import json
base_path = 'gs://airport-ops/csv/'
b = db.read_text(base_path+'Complaints_Suggestions_Compliments_2022.csv', blocksize="1024MiB")
b.take(3)

('\ufeffCase Number,Case Origin,Issue Terminal,Case Record Type,Case Type,Category (F),Sub-Category (F),Issue Category,Subject,Description,Date/Time Opened,Case Owner\n',
 '3033230,Web,T1,Feedback,Complaint,Ground Transport & Roadway,"Transport, Trolley, Carpark",Buggy,,The buggy car was speeding in front of the currency exchange around 23.07 in terminal 1.,22/10/2022 23:12,Buggy Portfolio\n',
 '3010035,Web,T1,Feedback,Suggestion,Ground Transport & Roadway,"Transport, Trolley, Carpark",Buggy,-,"Hi Changi Airport, Me and my family of five (2 elderly parents, wife and 4-year-old daughter) recently departed from T1 to Jakarta. The distance from the immigration to the boarding gate (Scoot flight) is about 12 mins walk. As my father has difficulties walking long distances and uses a walking stick, a buggy service would have been helpful for him. We approached one of the ground staff at the airport but were informed we needed to book the service a few days in advance. Just to share that when

In [5]:
def standardize(record):
    import csv, sys
    from datetime import datetime
    csv.field_size_limit(sys.maxsize)
    reader = csv.reader([str(record)])
    
    for parsed_record in reader:
        case_number=parsed_record[0]
        case_origin=parsed_record[1]
        terminal   =parsed_record[2]
        case_record_type=parsed_record[3]
        case_type  =parsed_record[4]
        category   =parsed_record[5]
        sub_category=parsed_record[6]
        issue_category=parsed_record[7]
        subject=parsed_record[8]
        description=parsed_record[9]
        case_owner=parsed_record[11]
        parse_issue = 'Parsed successfully'
        opened_time=''
        # 22/10/2022 23:12
        try:
            opened_time = datetime.strptime(parsed_record[10], '%d/%m/%Y %H:%M')
            parse_issue = 'Parsed successfully'
        except:
            parse_issue=f'issue parsing opened time {parsed_record[10]}'
            try:
                opened_time = datetime.strptime(parsed_record[10], '%d/%m/%y %H:%M')
                parse_issue = 'Parsed successfully'
            except:
                parse_issue=f'issue parsing opened time {parsed_record[10]}'            
            
        return {
            'case_number': case_number,
            'case_origin': case_origin,
            'terminal': terminal,
            'case_record_type': case_record_type,
            'case_type': case_type,
            'category': category,
            'sub_category': sub_category,
            'issue_category': issue_category,
            'subject': subject,
            'description': description,
            'opened_time': opened_time,
            'case_owner': case_owner,
            'parse_issue': parse_issue
        }
    return {}

def isHeader(record):
    if  'Case Number' in record['case_number'] \
     and 'Case Origin' in record['case_origin']:
        return True
    return False

def hasParseError(record):
    if  'Parsed successfully' in record['parse_issue']:
        return False
    return True

final_df=b.map(standardize).remove(isHeader).remove(hasParseError)\
            .to_dataframe()\
            .astype({'case_number':'string','case_origin':'string','terminal':'string','case_record_type':'string','case_type':'string',
                    'category':'string', 'sub_category':'string', 'issue_category':'string', 'subject':'string', 'description':'string',
                    'opened_time':'M8[us]', 'case_owner':'string', 'parse_issue':'string'}).rename(columns={"case_number": "ID"}) 


In [6]:
len(final_df)

5283

In [7]:
len(final_df.ID.unique())

5283

#### Case Origin as an edge

In [8]:
co_df=final_df.case_origin.unique().to_frame().rename(columns={'case_origin':'ID'})

In [9]:
co_df.head()

,ID
0,Web
1,Phone
2,Email (Feedback)
3,Email (Enquiries)
4,Walk-in


In [10]:
co_edges_df=final_df[['ID','case_origin']]

In [11]:
co_edges_df.head()

,ID,case_origin
0,3033230,Web
1,3010035,Web
2,3007067,Web
3,3002588,Web
4,2993549,Web


#### Terminal as an edge

In [12]:
terminal_df=final_df.terminal.unique().to_frame().rename(columns={'terminal':'ID'})
terminal_edges_df=final_df[['ID','terminal']]

In [13]:
terminal_df.head()

,ID
0,T1
1,T3
2,CAC
3,General
4,T4


In [14]:
terminal_edges_df.head()

,ID,terminal
0,3033230,T1
1,3010035,T1
2,3007067,T3
3,3002588,T1
4,2993549,T1


#### Case Record Type as an edge "case_record_type"

In [15]:
cr_df=final_df.case_record_type.unique().to_frame().rename(columns={'case_record_type':'ID'})
cr_edges_df=final_df[['ID','case_record_type']]

In [16]:
cr_df.head()

,ID
0,Feedback


In [17]:
cr_edges_df.head()

,ID,case_record_type
0,3033230,Feedback
1,3010035,Feedback
2,3007067,Feedback
3,3002588,Feedback
4,2993549,Feedback


#### Case Type as an edge "case_type"

In [18]:
ct_df=final_df.case_type.unique().to_frame().rename(columns={'case_type':'ID'})
ct_edges_df=final_df[['ID','case_type']]

In [19]:
ct_df.head()

,ID
0,Complaint
1,Suggestion
2,Compliment


In [20]:
ct_edges_df.head()

,ID,case_type
0,3033230,Complaint
1,3010035,Suggestion
2,3007067,Complaint
3,3002588,Complaint
4,2993549,Suggestion


#### Issue Category as an edge   "issue_category"

In [21]:
it_df=final_df.issue_category.unique().to_frame().rename(columns={'issue_category':'ID'})
it_edges_df=final_df[['ID','issue_category']]

In [22]:
it_df.head()

,ID
0,Buggy
1,Changi Airfreight Centre & Cargo
2,Late Baggage Presentation
3,Check-in/ECI
4,Check-in/ ECI


In [23]:
it_edges_df.head()

,ID,issue_category
0,3033230,Buggy
1,3010035,Buggy
2,3007067,Buggy
3,3002588,Buggy
4,2993549,Buggy


#### Case Owner as an edge "case_owner"

In [24]:
cown_df=final_df.case_owner.unique().to_frame().rename(columns={'case_owner':'ID'})
cown_edges_df=final_df[['ID','case_owner']]

In [25]:
cown_df.head()

,ID
0,Buggy Portfolio
1,[Inactive]Air Cargo Portfolio
2,[Inactive]T1 Airline& Check-in Portfolio
3,[Inactive]P2P Transport Ops Portfolio
4,[Inactive]Ground Tpt(Coach & Shuttle) Pf


In [26]:
cown_edges_df.head()

,ID,case_owner
0,3033230,Buggy Portfolio
1,3010035,Buggy Portfolio
2,3007067,Buggy Portfolio
3,3002588,Buggy Portfolio
4,2993549,Buggy Portfolio


#### Category and sub category "category" and "sub_category"

In [27]:
cat_df=final_df.category.unique().to_frame().rename(columns={'category':'ID'})
subcat_df=final_df.sub_category.unique().to_frame().rename(columns={'sub_category':'ID'})
subcat_edges_df=final_df[['ID','sub_category']]

In [28]:
cat_subcat_edges_df=final_df[['category','sub_category']].drop_duplicates(subset=['category','sub_category'])

In [29]:
cat_subcat_edges_df.head()

,category,sub_category
0,Ground Transport & Roadway,"Transport, Trolley, Carpark"
6,"Airside, Baggage, Cargo Complex",Changi Airfreight Centre & Cargo
15,"Airside, Baggage, Cargo Complex",Baggage
16,"Check-in & Airline, Immigration & Transfer","Check-in, Boarding & Airline"
19,Commercial Services (CD),Operations (C)


In [30]:
subcat_edges_df.head()

,ID,sub_category
0,3033230,"Transport, Trolley, Carpark"
1,3010035,"Transport, Trolley, Carpark"
2,3007067,"Transport, Trolley, Carpark"
3,3002588,"Transport, Trolley, Carpark"
4,2993549,"Transport, Trolley, Carpark"


#### Drop columns that are now edges

In [31]:
final_df=final_df.drop(columns=['case_origin', 'terminal','case_record_type','case_type','issue_category',
                                'case_owner','category','sub_category', 'parse_issue'])  

#### Process words and TF-IDF score

In [32]:
w = db.read_text(base_path+'edges_to_words.csv', blocksize="1024MiB")
w.take(3)

('buggy,3033230,0.3813825246578165\n',
 'car,3033230,0.2884718494335256\n',
 'currency,3033230,0.3997072609216392\n')

In [34]:
def standardize_tf(record):
    import csv, sys
    from datetime import datetime
    csv.field_size_limit(sys.maxsize)
    reader = csv.reader([str(record)])
    
    for parsed_record in reader:
        word=parsed_record[0]
        case_number=parsed_record[1]
        score = 0.0
        try:
            score   = float(parsed_record[2])
            parse_issue = 'Parsed successfully'
        except:
            parse_issue=f'issue parsing score {parsed_record[2]}'
            
        return {
            'word': word,
            'case_number': case_number,
            'score': score,
            'parse_issue': parse_issue
        }
    return {}


def hasParseError_tf(record):
    if  'Parsed successfully' in record['parse_issue']:
        return False
    return True

edges_to_words_df=w.map(standardize_tf).remove(hasParseError_tf)\
            .to_dataframe()\
            .astype({'word':'string','case_number':'string','score':'float','parse_issue':'string'})

In [35]:
edges_to_words_df.head()

,word,case_number,score,parse_issue
0,buggy,3033230,0.381383,Parsed successfully
1,car,3033230,0.288472,Parsed successfully
2,currency,3033230,0.399707,Parsed successfully
3,exchange,3033230,0.367169,Parsed successfully
4,speeding,3033230,0.502225,Parsed successfully


In [36]:
len(edges_to_words_df)

107361

In [38]:
words_df=edges_to_words_df.word.unique().to_frame().rename(columns={'word':'ID'})

In [39]:
words_df.head()

,ID
0,buggy
1,car
2,currency
3,exchange
4,speeding


### Import graph

In [41]:
rc=remote.Client()
rc.server_version

'0.5.0+20221106T212601Z.9c14e060f.dev'

In [42]:
graph = rc.create_graph(num_partitions=2)
graph.graph_id

'G9JaDmZQ1V9Cfn9zDgJ8SWCxQGWCWe9jn5JygvCw2GEK'

In [43]:

with import_data.DataFrameImporter(graph) as df_importer:
    df_importer.nodes_dataframe(final_df, id_column="ID", id_space='CASE', label='CASE')
    df_importer.nodes_dataframe(co_df, id_column="ID", id_space='CASE_ORIGIN', label='CASE_ORIGIN')
    df_importer.nodes_dataframe(terminal_df, id_column="ID", id_space='TERMINAL', label='TERMINAL')
    df_importer.nodes_dataframe(cr_df, id_column="ID", id_space='CASE_RECORD_TYPE', label='CASE_RECORD_TYPE')
    df_importer.nodes_dataframe(ct_df, id_column="ID", id_space='CASE_TYPE', label='CASE_TYPE')
    df_importer.nodes_dataframe(it_df, id_column="ID", id_space='ISSUE_CATEGORY', label='ISSUE_TYPE')
    df_importer.nodes_dataframe(cown_df, id_column="ID", id_space='CASE_OWNER', label='CASE_OWNER')
    df_importer.nodes_dataframe(cat_df, id_column="ID", id_space='CATEGORY', label='CATEGORY')
    df_importer.nodes_dataframe(subcat_df, id_column="ID", id_space='SUB_CATEGORY', label='SUB_CATEGORY')
    df_importer.nodes_dataframe(words_df, id_column="ID", id_space='WORD', label='WORD')
#
    df_importer.edges_dataframe(co_edges_df, source_column='case_origin', destination_column='ID',\
                                source_id_space='CASE_ORIGIN', destination_id_space='CASE', \
                                type='CASE_ORIGIN_FOR')
    df_importer.edges_dataframe(terminal_edges_df, source_column='terminal', destination_column='ID',\
                                source_id_space='TERMINAL', destination_id_space='CASE', \
                                type='TERMINAL_FOR')
    df_importer.edges_dataframe(cr_edges_df, source_column='case_record_type', destination_column='ID',\
                                source_id_space='CASE_RECORD_TYPE', destination_id_space='CASE', \
                                type='CASE_RECORD_TYPE_FOR')
    df_importer.edges_dataframe(ct_edges_df, source_column='case_type', destination_column='ID',\
                                source_id_space='CASE_TYPE', destination_id_space='CASE', \
                                type='CASE_TYPE_FOR')
    df_importer.edges_dataframe(it_edges_df, source_column='issue_category', destination_column='ID',\
                                source_id_space='ISSUE_CATEGORY', destination_id_space='CASE', \
                                type='ISSUE_CATEGORY_FOR')
    df_importer.edges_dataframe(cown_edges_df, source_column='case_owner', destination_column='ID',\
                                source_id_space='CASE_OWNER', destination_id_space='CASE', \
                                type='OWNS_CASE')
    df_importer.edges_dataframe(cat_subcat_edges_df, source_column='category', destination_column='sub_category',\
                                source_id_space='CATEGORY', destination_id_space='SUB_CATEGORY', \
                                type='BELONGS_TO_SUB_CATEGORY')
    df_importer.edges_dataframe(subcat_edges_df, source_column='sub_category', destination_column='ID',\
                                source_id_space='SUB_CATEGORY', destination_id_space='CASE', \
                                type='SUB_CATEGORY_FOR')
    df_importer.edges_dataframe(edges_to_words_df, source_column='word', destination_column='case_number',\
                                source_id_space='WORD', destination_id_space='CASE', \
                                type='SAID_IN_CASE')
    df_importer.insert()

          0/? [?op/s]

          0/? [?op/s]

In [44]:
query="""
MATCH  (c:CASE)<-[s:SAID_IN_CASE]-(w:WORD)-[:SAID_IN_CASE]->(c1:CASE)
where s.score > 0.5
  and c.ID <> c1.ID
return c,w,c1 limit 100
"""
graph.query(query, contextualize=True).view()

          0/? [?op/s]

          0/? [?op/s]

In [46]:
query="""
MATCH  (c:CASE)<-[s:SAID_IN_CASE]-(w:WORD)-[:SAID_IN_CASE]->(c1:CASE)
where s.score > 0.5
  and w.ID = 'recording'
return c,w,c1
"""
graph.query(query, contextualize=True).view()

          0/? [?op/s]

          0/? [?op/s]

In [53]:
query="""
match(y:CASE_TYPE)
where y.ID = 'Complaint'
MATCH(x)-[]-(y)
return x,y limit 100
"""
graph.query(query, contextualize=True).view()

          0/? [?op/s]

          0/? [?op/s]

In [125]:
query="""
MATCH(x)<-[]-(y:SUB_CATEGORY)<-[]-(z:CATEGORY)
return x,y,z limit 100
"""
graph.query(query, contextualize=True).view()

          0/? [?op/s]

          0/? [?op/s]

### Promote Category to a Node

In [13]:
query="""
MATCH(c:CASE)
with distinct c.category as category
CREATE(cat:CATEGORY)
set cat.category=category
"""
graph.query(query).view()

          0/? [?op/s]

In [14]:
# link categories : check
query="""
MATCH (cas:CASE),(cat:CATEGORY)
WHERE EXISTS(cas.category)
  and cas.category = cat.category
return cas.ID, cas.category,id(cat), cat.category order by cas.ID limit 10
"""
graph.query(query).view()


          0/? [?op/s]

In [15]:
# link categories: establish
query="""
MATCH (cas:CASE),(cat:CATEGORY)
WHERE EXISTS(cas.category)
  and cas.category = cat.category
CREATE (cas)-[:HAS_CATEGORY]->(cat)
"""
graph.query(query).view()


          0/? [?op/s]

In [13]:
# link categories : check
query="""
MATCH (cas:CASE)-[:HAS_CATEGORY]->(cat:CATEGORY)
return cas, cat limit 10"""
graph.query(query, contextualize=True).view()


          0/? [?op/s]

In [47]:
graph.schema().view()

          0/? [?op/s]

In [28]:
def udf_pre_processing(g):
    from katana_enterprise.ai.preprocessing.feature_generator import BagOfWords
    from katana_enterprise.distributed import single_host
    from mpi4py import MPI
    import numpy
    import torch
    from torch_geometric.data import Data
    from torch_geometric.loader import DataLoader

    comm = MPI.COMM_WORLD
    corpus=[]
    data=None



    edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
    x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

    g_data = Data(x=x, edge_index=edge_index.t().contiguous())
    # print(g_data)
    data_list = [g_data, g_data, g_data]
    loader = DataLoader(data_list, batch_size=32)
    print(loader)
    corpus = numpy.array(
            [
                "This is a good document",
                "Paper relates to something in Computer Science",
                "Nothing is true",
                "Everything is permitted",
                "   ",
            ]
        )
    test_corpus = numpy.array(["All is well", "Nothing is true", "",])

    encoder = BagOfWords()
    encoder.fit(corpus)
    data = encoder.transform(test_corpus)
    # print(data)

    return single_host(data)

result = graph.run(udf_pre_processing)

          0/? [?op/s]


Host 0 output:

Host 1 output:


In [22]:
result

array([[0., 6., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        2.],
       [0., 6., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]])

In [25]:
type(result[0][0])

numpy.float64